This code was written to analyse the E. coli model with and without constriants.
We started by loading the model and optimizing to get the fluxes of the model
We then applied the constraint in different conditions:

Ec = WT + Ecoli (the both of them)

E = Ecoli alone

W = WT + Ecoli (LATE)

K = dKil + Ecoli

In [41]:
#importing the needed packages to study the model
import cobra
import pandas as pd

In [42]:
#loading mdoel
model = cobra.io.read_sbml_model("iML1515.xml")

After having loaded the model, we want to optimize it with no constraints to see how the model works and the fluxes of the model 

In [43]:
model_optimal =model.optimize()
model_optimal

,fluxes,reduced_costs
CYTDK2,0.000000e+00,9.521763e-17
XPPT,-8.698806e-17,1.387779e-17
HXPRT,0.000000e+00,-1.504671e-02
NDPK5,0.000000e+00,3.855759e-16
SHK3Dr,3.342403e-01,-2.639676e-18
...,...,...
MPTS,0.000000e+00,-2.602085e-18
MOCOS,0.000000e+00,-5.642514e-03
BMOGDS2,0.000000e+00,-3.338488e-01
FESD2s,0.000000e+00,-3.447763e-17


In [44]:
model_optimal_fluxes= model_optimal.fluxes
model_optimal_fluxes

CYTDK2     0.000000e+00
XPPT      -8.698806e-17
HXPRT      0.000000e+00
NDPK5      0.000000e+00
SHK3Dr     3.342403e-01
               ...     
MPTS       0.000000e+00
MOCOS      0.000000e+00
BMOGDS2    0.000000e+00
FESD2s     0.000000e+00
OCTNLL     0.000000e+00
Name: fluxes, Length: 2712, dtype: float64

In [45]:
model1 = cobra.io.read_sbml_model("iML1515.xml")
def Eflux(model,dict_expression,default_value):

    """Function used to do the Eflux

    

    Args:

        model (cobra.core.model.Model): A cobrapy model

        dict_expression (dict): Dict of normalized RNAseq result, the key is the gene ID and the value is the expression level

        default_value (int): Default value for the upper bound, it will be scaled with the expression level.

    

    """

    rlist = [] # list used to work with reaction with two genes
    lp = 0.9 # not useful anymore (At first I was also using the lower bound)
    up = 1.1 # add 10% to the upper bound
    for gene in model.genes: 
        if gene.id in dict_expression.keys(): # If the gene is in the RNA-seq data
            #print(f"gene : {gene}")
            #print(f"LB = dv({dv}) * lp({lp}) * de({dict_exp[gene.id]})")
            #lb = dv * lp * dict_exp[gene.id]
            #print(f"UB = dv({dv}) * lp({up}) * de({dict_exp[gene.id]})\n")
            ub = default_value * up * dict_expression[gene.id] # The new upper bound is calculated for this gene
            for react in gene.reactions: # For each reaction linked to this gene
                if react in model.exchanges:
                    continue
                if react.id in rlist: # If another gene is linked to it:
                    if react.upper_bound < ub: # use the maximum upper bound
                        react.upper_bound = ub
                        if react.lower_bound < 0: # and if it's reversible set the lower bound
                            react.lower_bound = -ub
                else: # If there is no other gene (or if it's the first time)
                    if react.lower_bound < 0: # if the reaction is reversible :
                        react.lower_bound = -ub # set the lower bound
                    react.upper_bound = ub # set the upper bound
                    rlist.append(react.id) #add the react in the list to see if there is another gene


In [46]:
def dict_maker(normalized_counts):
    df = pd.DataFrame(normalized_counts)
    df1 = df.T
    df1.columns = df1.iloc[0]
    df1 = df1.tail(-1)
    normalized_dict= df1.to_dict("list")
    return normalized_dict

In [47]:
normalized_table_W = pd.read_csv("/home/lynn/Desktop/internship/modelling/test/normalized_counts_W.csv")
normalized_dict_W = dict_maker(normalized_table_W)
normalized_table_Ec = pd.read_csv("/home/lynn/Desktop/internship/modelling/test/normalized_counts_Ec.csv")
normalized_dict_Ec = dict_maker(normalized_table_Ec)
normalized_table_E = pd.read_csv("/home/lynn/Desktop/internship/modelling/test/normalized_counts_E.csv")
normalized_dict_E = dict_maker(normalized_table_E)
normalized_table_E_C1 = pd.read_csv("/home/lynn/Desktop/internship/modelling/test/normalized_counts_E_C1.csv")
normalized_dict_E_C1 = dict_maker(normalized_table_E_C1)
normalized_table_E_C2 = pd.read_csv("/home/lynn/Desktop/internship/modelling/test/normalized_counts_E_C2.csv")
normalized_dict_E_C2 = dict_maker(normalized_table_E_C2)

In [48]:
for i in normalized_dict_W:
    normalized_dict_W[i]=normalized_dict_W[i][0]

In [49]:
normalized_dict_W

{'b0001': 3.32443715608506,
 'b0002': 7.99356638420409,
 'b0003': 4.52628062180678,
 'b0004': 5.27034922552012,
 'b0005': 4.16886557357548,
 'b0006': 5.32910254899284,
 'b0007': 5.66019619785197,
 'b0008': 9.87579541864754,
 'b0009': 5.97292182950196,
 'b0010': 6.34021245536015,
 'b0011': 6.38228453732756,
 'b0012': 2.96084307871541,
 'b0013': 3.32443715608506,
 'b0014': 9.39996145365677,
 'b0015': 5.13209966618183,
 'b0016': 5.58011385242465,
 'b0018': 2.96084307871541,
 'b4412': 2.96084307871541,
 'b0019': 8.11800788091822,
 'b0020': 4.16886557357548,
 'b0021': 2.96084307871541,
 'b0022': 2.96084307871541,
 'b0023': 8.30511635450959,
 'b0024': 7.55094020022363,
 'b0025': 6.76700787208907,
 'b0026': 10.4451493807398,
 'b0027': 7.52887578432706,
 'b0028': 7.36354210096031,
 'b0029': 7.77185139377067,
 'b0030': 2.96084307871541,
 'b0031': 5.99811804335608,
 'b0032': 10.3510938266312,
 'b0033': 12.0850972964173,
 'b0034': 5.76924873118481,
 'b0035': 2.96084307871541,
 'b0036': 2.96084307

In [50]:
W = Eflux(model1, normalized_dict_W, 10)
model.optimal_W = model1.optimize()
model.optimal_W

,fluxes,reduced_costs
CYTDK2,0.000000,5.024293e-17
XPPT,0.000000,1.387779e-17
HXPRT,0.000000,-1.504672e-02
NDPK5,0.000000,-0.000000e+00
SHK3Dr,0.334241,2.915642e-17
...,...,...
MPTS,0.000000,8.673617e-18
MOCOS,0.000000,-5.642521e-03
BMOGDS2,0.000000,-3.338492e-01
FESD2s,0.000000,-1.951564e-17


In [51]:
for i in normalized_dict_Ec:
    normalized_dict_Ec[i]=normalized_dict_Ec[i][0]
normalized_dict_Ec

{'b0001': 2.43295940727611,
 'b0002': 8.6390150479401,
 'b0003': 7.51727569320958,
 'b0004': 7.57258654938577,
 'b0005': 4.10685079645022,
 'b0006': 6.20515788392009,
 'b0007': 3.96532254836725,
 'b0008': 9.51451681645401,
 'b0009': 6.97498811199193,
 'b0010': 5.32912359629157,
 'b0011': 1.22032995487956,
 'b0012': 0.0029952905485701,
 'b0013': 2.06004738366994,
 'b0014': 6.49920809085669,
 'b0015': 6.12163711829091,
 'b0016': 5.51159454153741,
 'b0018': 1.25096157353322,
 'b4412': 0.0029952905485701,
 'b0019': 7.97670731184688,
 'b0020': 5.56559717585422,
 'b0021': 3.67919857056692,
 'b0022': 5.34553831007001,
 'b0023': 8.92534673617656,
 'b0024': 7.06834836066393,
 'b0025': 8.2553118863596,
 'b0026': 10.8062837742865,
 'b0027': 6.65692533979425,
 'b0028': 7.86851392389546,
 'b0029': 8.62003649809878,
 'b0030': 4.49505552836802,
 'b0031': 6.78096619757708,
 'b0032': 12.1945383409723,
 'b0033': 13.4618771567868,
 'b0034': 7.07745665240059,
 'b0035': 2.31903981556254,
 'b0036': 1.189033

In [52]:
Ec = Eflux(model1, normalized_dict_Ec, 10)
model.optimal_Ec = model1.optimize()
model.optimal_Ec

,fluxes,reduced_costs
CYTDK2,0.000000,2.248735e-17
XPPT,0.000000,0.000000e+00
HXPRT,0.000000,-1.504672e-02
NDPK5,0.000000,-0.000000e+00
SHK3Dr,0.334241,-6.647183e-17
...,...,...
MPTS,0.000000,0.000000e+00
MOCOS,0.000000,-5.642521e-03
BMOGDS2,0.000000,-3.338492e-01
FESD2s,0.000000,-6.288373e-18


In [53]:
for i in normalized_dict_E:
    normalized_dict_E[i]=normalized_dict_E[i][0]
normalized_dict_E

{'b0001': 3.81352468929781,
 'b0002': 8.87578006306849,
 'b0003': 7.56711948795167,
 'b0004': 7.69564589909445,
 'b0005': 4.75114232472695,
 'b0006': 6.32930302481501,
 'b0007': 6.01502470527315,
 'b0008': 10.2625764465461,
 'b0009': 6.82017896241519,
 'b0010': 4.61999982982136,
 'b0011': 1.51096191927738,
 'b0012': 0.0058675423097269,
 'b0013': 3.72683121703249,
 'b0014': 8.56536869043088,
 'b0015': 7.50255337981306,
 'b0016': 6.82158267270679,
 'b0018': 2.75915583380027,
 'b4412': 0.0058675423097269,
 'b0019': 9.99332055016427,
 'b0020': 7.81595961821486,
 'b0021': 5.3387809438947,
 'b0022': 4.87233628810464,
 'b0023': 7.39583399636236,
 'b0024': 6.00090140274153,
 'b0025': 9.07660184861857,
 'b0026': 10.7899248312637,
 'b0027': 6.01145091173575,
 'b0028': 7.42877889133759,
 'b0029': 8.10831535538772,
 'b0030': 3.67694435910691,
 'b0031': 7.93928535647869,
 'b0032': 10.1131430888686,
 'b0033': 10.507735187255,
 'b0034': 5.92576211367192,
 'b0035': 1.6915341649192,
 'b0036': 0.9524635

In [54]:
E = Eflux(model1, normalized_dict_E, 10)
model.optimal_E = model1.optimize()
model.optimal_E

,fluxes,reduced_costs
CYTDK2,0.000000,2.248735e-17
XPPT,0.000000,0.000000e+00
HXPRT,0.000000,-1.504672e-02
NDPK5,0.000000,-0.000000e+00
SHK3Dr,0.334241,-6.647183e-17
...,...,...
MPTS,0.000000,0.000000e+00
MOCOS,0.000000,-5.642521e-03
BMOGDS2,0.000000,-3.338492e-01
FESD2s,0.000000,-6.288373e-18


In [55]:
for i in normalized_dict_E_C1:
    normalized_dict_E_C1[i]=normalized_dict_E_C1[i][0]
normalized_dict_E_C1

{'b0001': 3.31469652565629,
 'b0002': 8.21795769786411,
 'b0003': 7.03913839390696,
 'b0004': 7.22583403553784,
 'b0005': 4.30669970727279,
 'b0006': 5.84549005094437,
 'b0007': 3.63575439127776,
 'b0008': 9.62007316544424,
 'b0009': 7.15167619581708,
 'b0010': 4.99412711400313,
 'b0011': 0.787819021371666,
 'b0012': 0.0404636846081765,
 'b0013': 1.89141918684608,
 'b0014': 7.85337194086739,
 'b0015': 6.75595576955096,
 'b0016': 4.51096191927738,
 'b0018': 1.62714584316111,
 'b4412': 0.0404636846081765,
 'b0019': 7.68243284122705,
 'b0020': 5.16631374788993,
 'b0021': 3.47767732756531,
 'b0022': 3.38266725274504,
 'b0023': 9.17819175867861,
 'b0024': 6.76128527336162,
 'b0025': 7.31070365722511,
 'b0026': 11.4925992976279,
 'b0027': 6.42105494692424,
 'b0028': 7.16912328147676,
 'b0029': 8.0092127868052,
 'b0030': 3.38956681176273,
 'b0031': 6.54303182025524,
 'b0032': 12.4861964751488,
 'b0033': 13.6578509871448,
 'b0034': 7.21674585819531,
 'b0035': 1.52105073690096,
 'b0036': 0.7311

In [56]:
E_C1 = Eflux(model1, normalized_dict_E_C1, 10)
model.optimal_E_C1 = model1.optimize()
model.optimal_E_C1

,fluxes,reduced_costs
CYTDK2,0.000000,-4.497471e-17
XPPT,0.000000,0.000000e+00
HXPRT,0.000000,-1.665139e-02
NDPK5,0.000000,-0.000000e+00
SHK3Dr,0.315389,-3.683547e-17
...,...,...
MPTS,0.000000,9.801188e-17
MOCOS,0.000000,-8.325693e-03
BMOGDS2,0.000000,-4.131625e-01
FESD2s,0.000000,-2.289566e-02


In [57]:
for i in normalized_dict_E_C2:
    normalized_dict_E_C2[i]=normalized_dict_E_C2[i][0]
normalized_dict_E_C2

{'b0001': 4.27202318906105,
 'b0002': 8.71932036403724,
 'b0003': 10.6913073301911,
 'b0004': 7.35499863931907,
 'b0005': 4.3743439890385,
 'b0006': 9.21977355089287,
 'b0007': 4.15588959451476,
 'b0008': 9.40830850889606,
 'b0009': 10.2744713400658,
 'b0010': 5.52042224852645,
 'b0011': 1.48334442967338,
 'b0012': 0.0375459484678048,
 'b0013': 2.10433665981474,
 'b0014': 8.01100356443267,
 'b0015': 10.3102719472978,
 'b0016': 5.84649331596456,
 'b0018': 1.52606881166759,
 'b4412': 0.0375459484678048,
 'b0019': 7.77623561482759,
 'b0020': 5.38370429247405,
 'b0021': 7.44418666269936,
 'b0022': 5.01346225980656,
 'b0023': 9.66204523878578,
 'b0024': 11.2438225742894,
 'b0025': 8.1811025507538,
 'b0026': 10.4050137853593,
 'b0027': 9.73940934222919,
 'b0028': 7.45343526884372,
 'b0029': 7.98384867402245,
 'b0030': 7.19199684666977,
 'b0031': 6.81890167588564,
 'b0032': 12.1375163317101,
 'b0033': 16.8189011767246,
 'b0034': 7.36045207176887,
 'b0035': 2.02856915219677,
 'b0036': 4.940678

In [58]:
E_C2 = Eflux(model1, normalized_dict_E_C2, 10)
model.optimal_E_C2 = model1.optimize()
model.optimal_E_C2

,fluxes,reduced_costs
CYTDK2,0.000000e+00,-2.775558e-17
XPPT,0.000000e+00,-1.387779e-17
HXPRT,0.000000e+00,-1.504672e-02
NDPK5,2.161838e-18,-0.000000e+00
SHK3Dr,3.342407e-01,-5.243356e-17
...,...,...
MPTS,0.000000e+00,-5.030698e-17
MOCOS,0.000000e+00,-3.413725e-01
BMOGDS2,0.000000e+00,2.775558e-17
FESD2s,0.000000e+00,-5.811324e-16


In [68]:
fluxes_no_constraints = model_optimal.fluxes
fluxes_W = model.optimal_W.fluxes
fluxes_Ec = model.optimal_Ec.fluxes
fluxes_E = model.optimal_E.fluxes
fluxes_E_C1 = model.optimal_E_C1.fluxes
fluxes_E_C2 = model.optimal_E_C2.fluxes
all_fluxes = [fluxes_no_constraints, fluxes_W, fluxes_Ec, fluxes_E, fluxes_E_C1, fluxes_E_C2]

In [69]:
df = pd.DataFrame(all_fluxes)
df

,CYTDK2,XPPT,HXPRT,NDPK5,SHK3Dr,NDPK6,NDPK8,DHORTS,OMPDC,PYNP2r,...,COLIPAabctex,ENLIPAabctex,DM_mththf_c,PYROX,BMOGDS1,MPTS,MOCOS,BMOGDS2,FESD2s,OCTNLL
fluxes,0.0,-8.698806e-17,0.0,0.000000e+00,0.334240,0.0,5.929692e-16,-0.290106,0.290106,2.320660e-16,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fluxes,0.0,0.000000e+00,0.0,0.000000e+00,0.334241,0.0,0.000000e+00,-0.290107,0.290107,0.000000e+00,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fluxes,0.0,0.000000e+00,0.0,0.000000e+00,0.334241,0.0,0.000000e+00,-0.290107,0.290107,0.000000e+00,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fluxes,0.0,0.000000e+00,0.0,0.000000e+00,0.334241,0.0,0.000000e+00,-0.290107,0.290107,0.000000e+00,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fluxes,0.0,0.000000e+00,0.0,0.000000e+00,0.315389,0.0,0.000000e+00,-0.273744,0.273744,0.000000e+00,...,0.0,1.741763e-17,-2.831126e-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fluxes,0.0,0.000000e+00,0.0,2.161838e-18,0.334241,0.0,0.000000e+00,-0.290107,0.290107,-2.161838e-18,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
df = df.set_axis(['Fluxes no constraints', 'Fluxes W', 'Fluxes Ec', 'Fluxes E', 'Fluxes E C1','Fluxes E C2'], axis='index')
df

,CYTDK2,XPPT,HXPRT,NDPK5,SHK3Dr,NDPK6,NDPK8,DHORTS,OMPDC,PYNP2r,...,COLIPAabctex,ENLIPAabctex,DM_mththf_c,PYROX,BMOGDS1,MPTS,MOCOS,BMOGDS2,FESD2s,OCTNLL
Fluxes no constraints,0.0,-8.698806e-17,0.0,0.000000e+00,0.334240,0.0,5.929692e-16,-0.290106,0.290106,2.320660e-16,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fluxes W,0.0,0.000000e+00,0.0,0.000000e+00,0.334241,0.0,0.000000e+00,-0.290107,0.290107,0.000000e+00,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fluxes Ec,0.0,0.000000e+00,0.0,0.000000e+00,0.334241,0.0,0.000000e+00,-0.290107,0.290107,0.000000e+00,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fluxes E,0.0,0.000000e+00,0.0,0.000000e+00,0.334241,0.0,0.000000e+00,-0.290107,0.290107,0.000000e+00,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fluxes E C1,0.0,0.000000e+00,0.0,0.000000e+00,0.315389,0.0,0.000000e+00,-0.273744,0.273744,0.000000e+00,...,0.0,1.741763e-17,-2.831126e-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fluxes E C2,0.0,0.000000e+00,0.0,2.161838e-18,0.334241,0.0,0.000000e+00,-0.290107,0.290107,-2.161838e-18,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [103]:
df1 = df[df.columns[df.sum()>0.0]]
df1

,NDPK5,SHK3Dr,NDPK8,OMPDC,PYNP2r,G5SD,CS,ICDHyr,PPA,APRAUR,...,I2FE2SR,TYRL,THZPSN3,PAI2T,APG3PAT120,4HTHRK,APH120,PETNT181pp,BIOMASS_Ec_iML1515_core_75p37M,ENLIPAabctex
Fluxes no constraints,0.000000e+00,0.334240,5.929692e-16,0.290106,2.320660e-16,0.193865,6.913003,6.913003,3.070861,0.000391,...,0.000002,0.000196,0.000196,0.000393,1.731617e-17,0.000587,3.671980e-17,0.000000e+00,0.876997,0.000000e+00
Fluxes W,0.000000e+00,0.334241,0.000000e+00,0.290107,0.000000e+00,0.193865,6.911908,6.911908,3.070865,0.000391,...,0.000002,0.000196,0.000196,0.000393,0.000000e+00,0.000587,0.000000e+00,0.000000e+00,0.876998,0.000000e+00
Fluxes Ec,0.000000e+00,0.334241,0.000000e+00,0.290107,0.000000e+00,0.193865,6.911908,6.911908,3.070865,0.000391,...,0.000002,0.000196,0.000196,0.000393,0.000000e+00,0.000587,0.000000e+00,0.000000e+00,0.876998,0.000000e+00
Fluxes E,0.000000e+00,0.334241,0.000000e+00,0.290107,0.000000e+00,0.193865,6.911908,6.911908,3.070865,0.000391,...,0.000002,0.000196,0.000196,0.000393,0.000000e+00,0.000587,0.000000e+00,0.000000e+00,0.876998,0.000000e+00
Fluxes E C1,0.000000e+00,0.315389,0.000000e+00,0.273744,0.000000e+00,0.182931,7.875691,7.875691,2.897665,0.000369,...,0.000002,0.000185,0.000185,0.000371,0.000000e+00,0.000554,0.000000e+00,1.245407e-17,0.827535,1.741763e-17
Fluxes E C2,2.161838e-18,0.334241,0.000000e+00,0.290107,-2.161838e-18,0.193865,6.911908,6.911908,3.070865,0.000391,...,0.000002,0.000196,0.000196,0.000393,0.000000e+00,0.000587,0.000000e+00,0.000000e+00,0.876998,0.000000e+00


In [104]:
df1.to_csv("fluxes_results.csv")

In [105]:
df1_diff = pd.DataFrame(df1.iloc[3] - df1.iloc[1]).T
df1_diff

,NDPK5,SHK3Dr,NDPK8,OMPDC,PYNP2r,G5SD,CS,ICDHyr,PPA,APRAUR,...,I2FE2SR,TYRL,THZPSN3,PAI2T,APG3PAT120,4HTHRK,APH120,PETNT181pp,BIOMASS_Ec_iML1515_core_75p37M,ENLIPAabctex
0,0.0,3.441691e-15,0.0,2.886580e-15,0.0,1.915135e-15,4.174439e-14,3.996803e-14,-3.241851e-14,3.903128e-18,...,1.821121e-20,1.951564e-18,1.951564e-18,9.324139e-17,0.0,1.141665e-16,0.0,0.0,8.770762e-15,0.0


In [106]:
df2 = pd.concat([df1, df1_diff], ignore_index=False)
df2

,NDPK5,SHK3Dr,NDPK8,OMPDC,PYNP2r,G5SD,CS,ICDHyr,PPA,APRAUR,...,I2FE2SR,TYRL,THZPSN3,PAI2T,APG3PAT120,4HTHRK,APH120,PETNT181pp,BIOMASS_Ec_iML1515_core_75p37M,ENLIPAabctex
Fluxes no constraints,0.000000e+00,3.342403e-01,5.929692e-16,2.901063e-01,2.320660e-16,1.938646e-01,6.913003e+00,6.913003e+00,3.070861e+00,3.911408e-04,...,1.753994e-06,1.955704e-04,1.955704e-04,3.928948e-04,1.731617e-17,5.867111e-04,3.671980e-17,0.000000e+00,8.769972e-01,0.000000e+00
Fluxes W,0.000000e+00,3.342407e-01,0.000000e+00,2.901066e-01,0.000000e+00,1.938648e-01,6.911908e+00,6.911908e+00,3.070865e+00,3.911412e-04,...,1.753996e-06,1.955706e-04,1.955706e-04,3.928952e-04,0.000000e+00,5.867118e-04,0.000000e+00,0.000000e+00,8.769982e-01,0.000000e+00
Fluxes Ec,0.000000e+00,3.342407e-01,0.000000e+00,2.901066e-01,0.000000e+00,1.938648e-01,6.911908e+00,6.911908e+00,3.070865e+00,3.911412e-04,...,1.753996e-06,1.955706e-04,1.955706e-04,3.928952e-04,0.000000e+00,5.867118e-04,0.000000e+00,0.000000e+00,8.769982e-01,0.000000e+00
Fluxes E,0.000000e+00,3.342407e-01,0.000000e+00,2.901066e-01,0.000000e+00,1.938648e-01,6.911908e+00,6.911908e+00,3.070865e+00,3.911412e-04,...,1.753996e-06,1.955706e-04,1.955706e-04,3.928952e-04,0.000000e+00,5.867118e-04,0.000000e+00,0.000000e+00,8.769982e-01,0.000000e+00
Fluxes E C1,0.000000e+00,3.153892e-01,0.000000e+00,2.737444e-01,0.000000e+00,1.829307e-01,7.875691e+00,7.875691e+00,2.897665e+00,3.690805e-04,...,1.655069e-06,1.845402e-04,1.845402e-04,3.707356e-04,0.000000e+00,5.536207e-04,0.000000e+00,1.245407e-17,8.275347e-01,1.741763e-17
Fluxes E C2,2.161838e-18,3.342407e-01,0.000000e+00,2.901066e-01,-2.161838e-18,1.938648e-01,6.911908e+00,6.911908e+00,3.070865e+00,3.911412e-04,...,1.753996e-06,1.955706e-04,1.955706e-04,3.928952e-04,0.000000e+00,5.867118e-04,0.000000e+00,0.000000e+00,8.769982e-01,0.000000e+00
0,0.000000e+00,3.441691e-15,0.000000e+00,2.886580e-15,0.000000e+00,1.915135e-15,4.174439e-14,3.996803e-14,-3.241851e-14,3.903128e-18,...,1.821121e-20,1.951564e-18,1.951564e-18,9.324139e-17,0.000000e+00,1.141665e-16,0.000000e+00,0.000000e+00,8.770762e-15,0.000000e+00


In [107]:
df2 = df2.set_axis(['Fluxes no constraints', 'Fluxes W', 'Fluxes Ec', 'Fluxes E', 'Fluxes E C1','Fluxes E C2', 'FLuxes E-W'], axis='index')
df2

,NDPK5,SHK3Dr,NDPK8,OMPDC,PYNP2r,G5SD,CS,ICDHyr,PPA,APRAUR,...,I2FE2SR,TYRL,THZPSN3,PAI2T,APG3PAT120,4HTHRK,APH120,PETNT181pp,BIOMASS_Ec_iML1515_core_75p37M,ENLIPAabctex
Fluxes no constraints,0.000000e+00,3.342403e-01,5.929692e-16,2.901063e-01,2.320660e-16,1.938646e-01,6.913003e+00,6.913003e+00,3.070861e+00,3.911408e-04,...,1.753994e-06,1.955704e-04,1.955704e-04,3.928948e-04,1.731617e-17,5.867111e-04,3.671980e-17,0.000000e+00,8.769972e-01,0.000000e+00
Fluxes W,0.000000e+00,3.342407e-01,0.000000e+00,2.901066e-01,0.000000e+00,1.938648e-01,6.911908e+00,6.911908e+00,3.070865e+00,3.911412e-04,...,1.753996e-06,1.955706e-04,1.955706e-04,3.928952e-04,0.000000e+00,5.867118e-04,0.000000e+00,0.000000e+00,8.769982e-01,0.000000e+00
Fluxes Ec,0.000000e+00,3.342407e-01,0.000000e+00,2.901066e-01,0.000000e+00,1.938648e-01,6.911908e+00,6.911908e+00,3.070865e+00,3.911412e-04,...,1.753996e-06,1.955706e-04,1.955706e-04,3.928952e-04,0.000000e+00,5.867118e-04,0.000000e+00,0.000000e+00,8.769982e-01,0.000000e+00
Fluxes E,0.000000e+00,3.342407e-01,0.000000e+00,2.901066e-01,0.000000e+00,1.938648e-01,6.911908e+00,6.911908e+00,3.070865e+00,3.911412e-04,...,1.753996e-06,1.955706e-04,1.955706e-04,3.928952e-04,0.000000e+00,5.867118e-04,0.000000e+00,0.000000e+00,8.769982e-01,0.000000e+00
Fluxes E C1,0.000000e+00,3.153892e-01,0.000000e+00,2.737444e-01,0.000000e+00,1.829307e-01,7.875691e+00,7.875691e+00,2.897665e+00,3.690805e-04,...,1.655069e-06,1.845402e-04,1.845402e-04,3.707356e-04,0.000000e+00,5.536207e-04,0.000000e+00,1.245407e-17,8.275347e-01,1.741763e-17
Fluxes E C2,2.161838e-18,3.342407e-01,0.000000e+00,2.901066e-01,-2.161838e-18,1.938648e-01,6.911908e+00,6.911908e+00,3.070865e+00,3.911412e-04,...,1.753996e-06,1.955706e-04,1.955706e-04,3.928952e-04,0.000000e+00,5.867118e-04,0.000000e+00,0.000000e+00,8.769982e-01,0.000000e+00
FLuxes E-W,0.000000e+00,3.441691e-15,0.000000e+00,2.886580e-15,0.000000e+00,1.915135e-15,4.174439e-14,3.996803e-14,-3.241851e-14,3.903128e-18,...,1.821121e-20,1.951564e-18,1.951564e-18,9.324139e-17,0.000000e+00,1.141665e-16,0.000000e+00,0.000000e+00,8.770762e-15,0.000000e+00


In [119]:
sorted_row_E_W = df2.iloc[6].sort_values(ascending=False)
sorted_row_E_W

EX_co2_e    1.598721e-13
ENO         1.207923e-13
ATPS4rpp    9.947598e-14
PYK2        9.414691e-14
GAPD        6.750156e-14
                ...     
PItex      -1.298961e-13
PIt2rpp    -1.300071e-13
NH4tpp     -1.367795e-13
NH4tex     -1.367795e-13
EX_h2o_e   -2.273737e-13
Name: FLuxes E-W, Length: 454, dtype: float64

In [142]:
reaction_names = sorted_row_E_W.index[:15]
gene_list = []
for reaction in reaction_names:
    reaction_id = model1.reactions.get_by_id(reaction)
    genes = [gene.id for gene in reaction_id.genes]
    gene_list.extend(genes)

In [143]:
gene_list

['b2779',
 'b3732',
 'b3734',
 'b3738',
 'b3739',
 'b3733',
 'b3737',
 'b3731',
 'b3736',
 'b3735',
 'b1854',
 'b1779',
 'b0116',
 'b0114',
 'b0115',
 'b0429',
 'b0431',
 'b0432',
 'b0430',
 'b1276',
 'b0118',
 'b1276',
 'b0118',
 'b0720',
 'b1136',
 'b0724',
 'b0723',
 'b0721',
 'b0722',
 'b0726',
 'b0116',
 'b0727',
 'b3919',
 'b3386']